# Rio Grande do Sul COVID cases Data Analysis
---

In [2]:
import dateutil.utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
import re
import scipy
from unidecode import unidecode
import seaborn as sns

## Step 1: Merging all data-sets

In [5]:
d1 = pd.read_csv("../db/dados-rs-1.csv", delimiter=";", on_bad_lines='skip', encoding='latin1', low_memory=True)
d2 = pd.read_csv("../db/dados-rs-2.csv", delimiter=";", on_bad_lines='skip', encoding='latin1', low_memory=True)
d3 = pd.read_csv("../db/dados-rs-3.csv", delimiter=";", on_bad_lines='skip', encoding='latin1', low_memory=True)
d4 = pd.read_csv("../db/dados-rs-4.csv", delimiter=";", on_bad_lines='skip', encoding='latin1', low_memory=True)
d5 = pd.read_csv("../db/dados-rs-5.csv", delimiter=";", on_bad_lines='skip', encoding='latin1', low_memory=True)
d6 = pd.read_csv("../db/dados-rs-6.csv", delimiter=";", on_bad_lines='skip', encoding='latin1', low_memory=True)

df = pd.concat([d1, d2, d3, d4, d5, d6])
df.to_csv('dados-rs.csv', index=False)

# CBO - Classificação Brasileira de Ocupações

          ÿid           dataNotificacao        dataInicioSintomas  \
0  HE8B8WiBpk  2020-08-25T03:00:37.280Z  2020-08-24T03:00:00.000Z   
1  IfUV4kqF1I  2020-08-17T03:00:48.914Z  2020-08-03T03:00:00.000Z   
2  auCqY2SQ4w  2020-06-26T17:37:33.237Z  2020-06-10T03:00:00.000Z   
3  R8MGrEmw1z  2020-06-26T03:00:00.000Z  2020-06-26T03:00:00.000Z   
4  w6ClV1iTLg  2020-08-21T03:00:51.066Z  2020-08-11T03:00:00.000Z   

  dataNascimento       sintomas profissionalSaude  \
0      undefined         Outros               Não   
1      undefined         Outros               Não   
2      undefined         Outros               Não   
3      undefined  Tosse, Outros               Sim   
4      undefined         Outros               Não   

                                                 cbo  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3  5151 - Outro tipo de agente de saúde ou

## Step 2: Data Processing

In [ ]:
# Loading the dataset
df = pd.read_csv("dados-rs.csv", delimiter=",", on_bad_lines='skip', encoding='utf-8', low_memory=True)
print(df.head(5))

In [6]:
# Some of the columns have values different from null or NaN
# but they are defined as 'undefined'

# After the analysis, the columns that must be droped are:
#   CBO
#   PaisOrigem
#   estado
#   estadoIBGE
#   origem
#   excluido
#   validado
#   dataNascimento

# Dropping columns that we deem to be unserviceable to our data analysis
df.drop(columns=["cbo", "paisOrigem", "estado", "estadoIBGE", "origem", "excluido", "validado", "dataNascimento"],
        inplace=True)

# Some of the values are defined as 'undefined', which are not
# classified as null or NaN for the pandas function
# Then, we are going to change this value to 'null'
df.sexo.replace("Indefinido", "null")

# Changing the data type of some columns to datetime
df.dataNotificacao = pd.to_datetime(df.dataNotificacao, errors='coerce')
df.dataTeste = pd.to_datetime(df.dataTeste, errors='coerce')
df.dataEncerramento = pd.to_datetime(df.dataEncerramento, errors='coerce')
df.dataInicioSintomas = pd.to_datetime(df.dataInicioSintomas, errors='coerce')

# Some lines (44510 [pd.isnull(df.sintomas).sum()]) have no symptoms described (== 'null')
# then, we are going to drop those lines
df = df[pd.notnull(df.sintomas)]

# There are some inconsistencies into de age date (i.e. ages up to 320)
# Since the population in RS with +100 is less than 0.0009% of the total population
# we are going to drop those lines with 100+ years
df = df[df.idade <= 100]

# Replacing variations of 'Dispneia' to only the correct spelling
df.sintomas.replace(regex=['Dispineia', 'Dispinéia', 'Dispnéia', 'Dificuldade De Respirar'], value='Dispneia',
                    inplace=True)

# Replacing variants of 'Assintomático'
df.sintomas.replace(regex=['Outros: Paciente assintomático'], value='Assintomático', inplace=True)

# Replacing variants of 'Dor de Garganta'
df.sintomas.replace(regex=['Dor De Garganta'], value='Dor de Garganta', inplace=True)

# Listing the symptoms
sintomas = []

# Splitting symptom column values into a tuple of symptoms
for i in df.sintomas:
    sintoma = re.split(",|, ", str(i))
    for j in sintoma:
        if j.strip() not in sintomas and j.strip() != '':
            sintomas.append(j.strip())

# Create the new columns to the symptoms variants
aux = 4
for i in sintomas:
    df.insert(aux, i, 'False')
    aux += 1

# For each symptom list, modify the target column for each symptom
for index, row in df.iterrows():
    for sim in sintomas:
        if sim in row['sintomas']:
            df.at[index, sim] = 'True'

# Saving the dataset without the selected columns
df.to_csv('../db/dados-rs-clean.csv', index=False)

In [ ]:
# Loading the clean dataframe
df = pd.read_csv("../db/dados-rs-clean.csv", delimiter=",", on_bad_lines='skip', encoding='utf-8', low_memory=True)

In [3]:
# Sorting DataFrame by dataNotificacao
df.sort_values(by=['dataNotificacao'], inplace=True)
print(df.head(10))

FileNotFoundError: [Errno 2] No such file or directory: 'dados-rs.csv'

In [ ]:
# Saving sorted dataframe into a new .csv
df.to_csv('../db/dados-rs-clean-sorted.csv', index=False)

In [ ]:
# Loading the clean and sorted dataframe
df = pd.read_csv("../db/dados-rs-clean-sorted.csv", delimiter=",", on_bad_lines='skip', encoding='utf-8',
                 low_memory=True)

In [ ]:
# Converting date columns to Date (yyyy-MM-dd)
df.dataNotificacao = pd.to_datetime(df.dataNotificacao, errors='coerce').dt.date
df.dataTeste = pd.to_datetime(df.dataTeste, errors='coerce').dt.date
df.dataEncerramento = pd.to_datetime(df.dataEncerramento, errors='coerce').dt.date
df.dataInicioSintomas = pd.to_datetime(df.dataInicioSintomas, errors='coerce').dt.date

In [ ]:
# Dropping lines without 'dataNotificacao'
df = df[pd.notnull(df.dataNotificacao)]

In [ ]:
# creating new columns to describe week and year number
df.insert(31, 'WeekNumber', 'null')
df.insert(32, 'YearNumber', 'null')

df['WeekNumber'] = df.dataNotificacao.apply(lambda x: x.isocalendar()[1])
df['YearNumber'] = df.dataNotificacao.apply(lambda x: x.isocalendar()[0])

# Step 3: Data Analysis
## 26. Has there been a significant surge in the number of COVID-19 cases in dates close to public (national and state) holidays?

In [ ]:
def generateHollidayPlot(date1, date2, title):
    df1 = df.loc[(df.dataNotificacao >= date1) & (df.dataNotificacao <= date2)]
    fig = sns.histplot(df1.dataNotificacao, kde=True)
    fig.set_title(title)
    for item in fig.get_xticklabels():
        item.set_rotation(45)
        item.set_horizontalalignment = 'right'
    plt.savefig(f"../plots/{title.strip().lower().replace(' ', '_')}{date1.year}.png")
    plt.show()

In [ ]:
# Carnival (24/02) (26/02/2020)
generateHollidayPlot(date(2020, 2, 10), date(2020, 3, 18), 'Carnaval')

# Paixão de Cristo (10/04/2020)
generateHollidayPlot(date(2020, 3, 27), date(2020, 5, 1), 'Paixao de Cristo')

# Tiradentes (21/04/2020)
generateHollidayPlot(date(2020, 4, 7), date(2020, 5, 12), 'Tiradentes')

# Dia Mundial do Trabalho (01/05/2020)
generateHollidayPlot(date(2020, 4, 17), date(2020, 5, 22), 'Dia do Trabalho')

In [ ]:
# Courpus Christi (11/06/2020)
generateHollidayPlot(date(2020, 5, 28), date(2020, 7, 2), 'Corpus Christi')

In [ ]:
# Independencia do Brasil (07/09/2020)
generateHollidayPlot(date(2020, 8, 24), date(2020, 9, 28), 'Independencia do Brasil')

# Dia do Gaucho / Rev Farroupilha (20/09/2020)
generateHollidayPlot(date(2020, 9, 6), date(2020, 10, 11), 'Dia do Gaucho')

# Nossa Senhora Aparecida (12/10/2020)
generateHollidayPlot(date(2020, 9, 28), date(2020, 11, 2), 'Nossa Senhora Aparecida')

# Dia do Servidor Publico (28/10/2020)
generateHollidayPlot(date(2020, 10, 14), date(2020, 11, 18), 'Dia do servidor publico')

# Dia dos Finados (02/11/2020)
generateHollidayPlot(date(2020, 10, 19), date(2020, 11, 23), 'Finados')

# Proclamacao da republica (15/11/2020)
generateHollidayPlot(date(2020, 11, 1), date(2020, 12, 6), 'Proclamacao da Republica')

# Natal (25/12/2020)
generateHollidayPlot(date(2020, 12, 11), date(2021, 1, 15), 'Natal')

# Ano novo (01/01/2021)
generateHollidayPlot(date(2020, 12, 17), date(2021, 1, 22), 'Ano novo')

# Carnaval (15/02/2021)
generateHollidayPlot(date(2021, 2, 1), date(2021, 3, 10), 'Carnaval')

# Paixao de Cristo (02/04/2021)
generateHollidayPlot(date(2021, 3, 19), date(2021, 4, 23), 'Paixao de Cristo')

# Tirandentes (21/04/2021)
generateHollidayPlot(date(2021, 4, 7), date(2021, 5, 5), 'Tiradentes')

# Dia Mundial do Trabalho (21/04/2021)
generateHollidayPlot(date(2021, 4, 17), date(2021, 5, 22), 'Dia do Trabalho')

# Corpus Christi (03/06/2021)
generateHollidayPlot(date(2021, 5, 20), date(2021, 6, 24), 'Corpus Christi')

In [ ]:
df1 = df.loc[(df.dataNotificacao >= date(2020, 5, 28)) & (df.dataNotificacao <= date(2020, 11, 14))]
#df2 = df.loc[(df.dataNotificacao >= date(2020, 4, 24)) & (df.dataNotificacao <= date(2020, 5, 29))]

df2 = df1.groupby(df['WeekNumber'])['ÿid'].count()
df2.diff()
df2.plot(kind='line', figsize=(10, 5), legend=None)

In [ ]:
# General graph per year

def generateYearCasesGraphGroupByWeeks(yearNumber):
    dfA = df.loc[df.YearNumber == yearNumber]
    dfB = dfA.groupby(df['WeekNumber'])['ÿid'].count()
    dfB.diff()
    dfB.plot(kind='line', figsize=(10, 5), legend=None)

### Number of registered cases in 2020

In [ ]:
generateYearCasesGraphGroupByWeeks(2020)

### Number of registered cases in 2021

In [ ]:
generateYearCasesGraphGroupByWeeks(2021)

# Gráfico da diferença
# Colocar um ponto colorido em todo feriado
# +2 pontos nas duas proximas semanas depois do feriado, para ver se
# são de subida

# Calcular a média móvel 2 semans antes e 2 semanas depois e plotar em
# cima desse gráfico

### Generate the graph with the full dataframe (2020 + 2021)

In [ ]:
df1 = df.loc[df.YearNumber == 2020].copy()
df2 = df.loc[df.YearNumber == 2021].copy()
df2['WeekNumber'] = df2.WeekNumber.apply(lambda x: x + 53)
print(df1.shape)
print(df2.shape)
print(df2.tail(5))

In [ ]:
# Merging the 2020 dataframe with the 2021 dataframe
df1 = pd.concat([df1, df2])
df1.shape

In [ ]:
# Plotting the full dataframe graph
df2 = df1.groupby(df1['WeekNumber'])['ÿid'].count()
df2 = df2.diff()

fig, ax = plt.subplots(figsize=(30, 14))
plot = sns.lineplot(
    data=df2
)
plot.set(xlabel='Weeks', ylabel='New Cases Per Week', title='Total cases per week from January 2020 to June 2021')

feriados = {
    'Feriado': ["1. Carnaval 20", "2. Paixao Cristo 20", "3. Tiradentes 20", "4. Dia Trabalho 20",
                "5. Corpus Christi 20", "6. Independencia 20",
                "7. Dia Gaucho 20", "8. Dia Crianca 20", "9. ServidorPublico20", "10. Finados20",
                "11. Proclamacao Republica 20", "12. Natal 20",
                "13. Ano Novo 20", "14. Carnaval 21", "15. Paixao Cristo 21", "16. Tiradentes 21",
                "17. Dia Trabalho 21", "18. Corpus Christi 21"],
    'Data': [date(2020, 2, 25).isocalendar()[1], date(2020, 4, 10).isocalendar()[1], date(2020, 4, 21).isocalendar()[1],
             date(2020, 5, 1).isocalendar()[1], date(2020, 6, 11).isocalendar()[1], date(2020, 9, 7).isocalendar()[1],
             date(2020, 9, 20).isocalendar()[1], date(2020, 10, 12).isocalendar()[1],
             date(2020, 10, 28).isocalendar()[1], date(2020, 11, 2).isocalendar()[1],
             date(2020, 11, 15).isocalendar()[1], date(2020, 12, 25).isocalendar()[1],
             date(2020, 12, 31).isocalendar()[1], 53 + date(2021, 2, 16).isocalendar()[1],
             53 + date(2021, 4, 2).isocalendar()[1],
             53 + date(2021, 4, 21).isocalendar()[1], 53 + date(2021, 5, 1).isocalendar()[1],
             53 + date(2021, 6, 11).isocalendar()[1]
             ]
}

dff = pd.DataFrame(feriados)

sns.scatterplot(x=dff['Data'], y=0, s=0, hue=dff['Feriado'], palette='Paired')
sns.color_palette("Paired", 1)

for i, j in df2.items():
    ax.scatter(x=i, y=df2[i], s=10, c='#1F77B4')


# Fuction to add the Holliday points to the scatter plot
def addHolidayPoints(data):
    ax.scatter(x=data - 1, y=df2[data - 1], s=50, c='red')
    ax.annotate('', xy=(data - 1, df2[data - 1]))


# Adding the points
for value in feriados['Data']:
    addHolidayPoints(value)

plt.savefig(f"../plots/teste.png")

## 31. What are the most common symptoms by age group? Is there a pattern?

In [ ]:
# Load the dataframe
df = pd.read_csv("../db/dados-rs-clean-sorted.csv", delimiter=",", on_bad_lines='skip', encoding='utf-8',
                 low_memory=True)

In [ ]:
# Separate the population by age intervals
# Possibilities:
# 1. Using the same age classes as the IBGE's age pyramid (INSERT REFERENCE)
# 5 to 5 years
# = 19 classes

df_per_age = []

for i in range(19):
    df_per_age[i] = df.loc[df.idade >= i * 5 & df.idade < i * 5 + 5]

cases_with_symptoms_per_age = []

# vector of dictionaries (each one with all possible symptoms)
for i in range(19):
    cases_with_symptoms_per_age.append({
        "Outros": 0,
        "Tosse": 0,
        "Dor de Garganta": 0,
        "Febre": 0,
        "Dispneia": 0,
        "Assintomático": 0,
        "Dor de Cabeça": 0,
        "Coriza": 0,
        "Distúrbios Gustativos": 0,
        "Distúrbios Olfativos": 0,
    })

# filling up

# each age group within df_per_age
for i in range(19):
    total = df.loc[d]

    # each line in df_per_age
    for index, row in df_per_age[i].iterrows():
        # each symptom in the given line of the dictionary array
        for key, value in cases_with_symptoms_per_age[i].items():
            if row[key] == 'True':
                cases_with_symptoms_per_age[i][key] += 1

# 1: Using the square rule
print(df)

## 34. What is the most used COVID test in the state? Proportionally, what is the percentage of positive results? How about negative ones?

In [ ]:
# Load the dataframe
df = pd.read_csv("../db/dados-rs-clean-sorted.csv", delimiter=",", on_bad_lines='skip', encoding='utf-8',
                 low_memory=True)